In [1]:
import numpy as np
from combinations import generate_suit_size_distributions, contingency_table, check_lex_order_for, bounded_compositions
from pref_contingency_tables import PrefContingencyTable
from tqdm import tqdm
from sympy.utilities.iterables import multiset_permutations
from itertools import combinations
from multiperms import multinomial, index2multiperm, multiperm2index

### NB! Тут описан алгоритм кодирования факторных матриц

Будем считать, что масти размера $0$ не бывает, тогда количество мастей может быть переменным, от $1$ до $4$. В соответствии с этим факторные матрицы имеют переменное число строк, по числу мастей.

In [2]:
def is_sorted_for(a: np.array):
    for i in range(a.shape[0] - 1):
        if a[i] > a[i + 1]:
            return False
    return True

def get_suit_sizes(n_suits, n_hands=3, max_hand_size=10, max_suit_size=8, reduce=True):
    result = []
    for hand_size in range(1, min(max_hand_size, n_suits * max_suit_size // n_hands) + 1):
        for suit_sizes in bounded_compositions(n_hands * hand_size, n_suits, n_suits * [max_suit_size]):
            if np.min(suit_sizes) > 0:
                if not reduce or is_sorted_for(suit_sizes[1:]):
                    result.append(np.copy(suit_sizes))
    return np.vstack(result)

def get_contingency_matrices(suit_sizes: np.array, n_hands=3, hand_sizes=None, reduce_perms=True):
    '''
    Returns all contingency tables with row sums equal to suit_sizes and column sums equal to hands_sizes.
    If reduced_perms = true, then rows with equal sums are sorted lexicographically.
    '''
    matrices = []
    for i in range(suit_sizes.shape[0]):
        if hand_sizes is None:
            hands_sizes = n_hands * [suit_sizes[i].sum() // n_hands]
        else:
            hands_sizes= hand_sizes
        for table in contingency_table(suit_sizes[i], hands_sizes):
            if reduce_perms:
            # check if equal suit sizes are ordered lexicographically
                lex_flag = False
                for j in range(1, len(suit_sizes[i]) - 1):
                    if suit_sizes[i][j] == suit_sizes[i][j+1] and \
                    not check_lex_order_for(table[j], table[j + 1]):
                        lex_flag = True
                        break
                if lex_flag:
                    continue            
            matrices.append(np.copy(table)[None, :])
    return np.concatenate(matrices, axis=0)

In [3]:
total = 0 
total_reduced = 0
for h in range(1, 5):
    total += get_suit_sizes(h, reduce=False).shape[0]
    total_reduced += get_suit_sizes(h, reduce=True).shape[0]
print(total, total_reduced)

1560 438


In [55]:
np.cos(np.array([1j, 2.0635j, 3j, 4j]))

array([ 1.54308063-0.j,  4.00024374-0.j, 10.067662  -0.j, 27.30823284-0.j])

Сначала посмотрим на матрицы сдач в первоначальном состоянии (с прикупом):

In [4]:
mat_all = get_contingency_matrices(np.array([8, 8, 8, 8], dtype=np.int8)[None, :], hand_sizes=[10, 10, 10, 2])
mat_all_full = get_contingency_matrices(
    np.array([8, 8, 8, 8], dtype=np.int8)[None, :], hand_sizes=[10, 10, 10, 2], reduce_perms=False
)

За счёт удаления лишних перестановок мастей произсходит сокращение почти ровно в 6 раз:

In [5]:
mat_all.shape, mat_all_full.shape

((42084, 4, 4), (248358, 4, 4))

На этапе торговли порядок мастей имеет значение, поэтому надо будет учесть все возможные расклады $32$ карт.

In [6]:
deal_counts = []
for matrix in tqdm(mat_all_full):
    count = 1
    for row in matrix:
        count *= multinomial(row)
    deal_counts.append(count)

100%|████████████████████████████████| 248358/248358 [00:06<00:00, 40233.32it/s]


In [7]:
vals, cnts = np.unique(deal_counts, return_counts=True)
vals[-5:], cnts[-5:]

(array([442552320000, 497871360000, 590069760000, 663828480000,
        885104640000]),
 array([384,  36,  72,  72,  90]))

Максимально имеем почти триллион раскладов на одну матрицу, что потребует $40$ бит для кодирования, остальные $24$ надо употребить на саму матрицу. Две карты прикупа можно разбросать десятью способами по мастям: $4$ одной масти и $\binom 42 = 6$ разных мастей. Таким образом, последний столбец матрицы требует 4 бита для кодирования. Например:

|  code  | column    |
| :----: | :-----:   |
| 0000   | [2,0,0,0] | 
| 0001   | [0,2,0,0] | 
| 0010   | [0,0,2,0] | 
| 0111   | [0,0,0,2] | 
| 0011   | [0,0,1,1] | 
| 0101   | [0,1,0,1] | 
| 0110   | [0,1,1,0] | 
| 1001   | [1,0,0,1] | 
| 1010   | [1,0,1,0] |
| 1100   | [1,1,0,0] | 

Коды $0100$, $1000$, $1011$, $1101$, $1110$, $1111$ пока остаются свободными.

Из оставшейся матрицы размера $4\times 3$ достаточно хранить $6$ чисел из левой верхней подматрицы размера $3\times 2$, поскольку оставшиеся значения восстанавливаются по известным суммам строк и столбцов. Матрица содержит числа от $0$ до $8$, причём появление числа $8$ в строке означает, что все остальные значения строки равны нулю. Поэтому будем записывать числа $0$ до $7$ как есть (для этого нужно 3 бита); если же первые два числа в строке равны $(0, 8)$ или $(8, 0)$, то запишем их как $(2, 7)$ или $(7, 2)$ соответственно. Таким образом, все матрицы размера $4\times 3$ требуют для хранения $18$ бит, а матрицы с меньшим числом мастей — ещё меньше. 

In [13]:
np.random.seed(2482359)
m = mat_all_full[np.random.randint(248358)]
m

array([[4, 4, 0, 0],
       [0, 3, 4, 1],
       [1, 3, 3, 1],
       [5, 0, 3, 0]])

In [14]:
def check_matrix(matrix):
    pct = PrefContingencyTable(matrix)
    index = hash(pct)
    matrix_from_index = PrefContingencyTable.from_hash(index).matrix
    assert np.all(matrix == matrix_from_index), f"{matrix} != {matrix_from_index}"
    return index

In [15]:
pct = PrefContingencyTable(m)
PrefContingencyTable.from_hash(hash(pct))

♠ 4400
♣ 0341
♦ 1331
♥ 5030

In [16]:
def check_matrix_array(matrices):
    max_hash = 0
    for matrix in tqdm(matrices):
        max_hash = max(max_hash, check_matrix(matrix))
    return max_hash

In [17]:
check_matrix_array(mat_all)

100%|███████████████████████████████████| 42084/42084 [00:06<00:00, 6490.64it/s]


15271400

In [18]:
check_matrix_array(mat_all_full)

100%|█████████████████████████████████| 248358/248358 [00:37<00:00, 6600.69it/s]


15299748

### Схема кодирования

У матриц, которые возникают во время розыгрыша, может быть различное количество мастей, их размеров, а также размеров рук (в отличие от матриц первоначального этапа сдачи с жёстко фиксированными размерами рук и мастей). Поэтому требуется хранить также размеры мастей в количестве до $4$ штук, со значениями от $1$ до $8$. Максимально для этого понадобится $12$ бит, что вместе с $18$-ю битами для матрицы составляет $30$ бит. За границу четырёх байт лучше не вылезать (чтобы сведения о матрицах без раскладов умещались в 32-битный int). С этой целью выделим два младших бита для диспатчинга:

* $00$: матрица полного расклада с мастями размера $[8, 8, 8, 8]$ и руками длин $[10, 10, 10, 2]$; далее $4 + 18 + 40 = 62$ бита предназначены для кодирования последнего столбца, верхней левой подматрицы и самого расклада; всё вместе идеально влазит в $64$-битный int; 

* $01$: матрица розыгрыша $10$-го этапа с руками длин $[10, 10, 10]$; размеры мастей восстанавливаются по тем же $4$ битам, что и в предыдущем случае; биты с 25-го по 28-й бит показывают козырную масть или её отсутствие;

* $10$: матрица розыгрыша этапов $1$—$9$ с четырьмя мастями; далее $12 + 18 =30$ бит отводятся для размеров мастей и факторной матрицы; 33-й бит показывает наличие козыря;

* $11$:  матрица розыгрыша этапов $1$—$8$ с $<4$ мастями; следующие два бита содержат число мастей, потом снова размеры мастей сами матрицы; 33-й бит показывает наличие козыря.

Теперь посмотрим, сколько матриц для числа мастей от 1 до 4 с разными режимами сокращений, и проверим хеширование на них.

In [19]:
mat1 = get_contingency_matrices(get_suit_sizes(1))
mat1_full = get_contingency_matrices(get_suit_sizes(1), reduce_perms=False)
mat1_full_full = get_contingency_matrices(get_suit_sizes(1, reduce=False), reduce_perms=False)
print(mat1.shape, mat1_full.shape, mat1_full_full.shape)
print(check_matrix_array(mat1))
print(check_matrix_array(mat1_full))
print(check_matrix_array(mat1_full_full))

(2, 3) (2, 3) (2, 3)


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 3339.41it/s]


87


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 2077.42it/s]


87


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 4249.55it/s]

87


In [22]:
mat2 = get_contingency_matrices(get_suit_sizes(2))
mat2_full = get_contingency_matrices(get_suit_sizes(2), reduce_perms=False)
mat2_full_full = get_contingency_matrices(get_suit_sizes(2, reduce=False), reduce_perms=False)
print(mat2.shape, mat2_full.shape, mat2_full_full.shape)
print(check_matrix_array(mat2))
print(check_matrix_array(mat2_full))
print(check_matrix_array(mat2_full_full))

(232, 2, 3) (232, 2, 3) (232, 2, 3)


100%|███████████████████████████████████████| 232/232 [00:00<00:00, 4577.49it/s]


44923


100%|███████████████████████████████████████| 232/232 [00:00<00:00, 6761.72it/s]


44923


100%|███████████████████████████████████████| 232/232 [00:00<00:00, 8429.52it/s]

44923


In [23]:
mat2

array([[[1, 1, 0],
        [0, 0, 1]],

       [[1, 0, 1],
        [0, 1, 0]],

       [[0, 1, 1],
        [1, 0, 0]],

       ...,

       [[2, 0, 5],
        [3, 5, 0]],

       [[1, 1, 5],
        [4, 4, 0]],

       [[0, 2, 5],
        [5, 3, 0]]])

In [17]:
mat3 = get_contingency_matrices(get_suit_sizes(3))
mat3_full = get_contingency_matrices(get_suit_sizes(3), reduce_perms=False)
mat3_full_full = get_contingency_matrices(get_suit_sizes(3, reduce=False), reduce_perms=False)
print(mat3.shape, mat3_full.shape, mat3_full_full.shape)
print(check_matrix_array(mat3))
print(check_matrix_array(mat3_full))
print(check_matrix_array(mat3_full_full))

(13059, 3, 3) (15339, 3, 3) (26018, 3, 3)


100%|██████████████████████████████████| 13059/13059 [00:00<00:00, 15569.03it/s]


30474239


100%|██████████████████████████████████| 15339/15339 [00:00<00:00, 15786.21it/s]


30605311


100%|██████████████████████████████████| 26018/26018 [00:01<00:00, 15958.40it/s]

30605311


In [18]:
mat4 = get_contingency_matrices(get_suit_sizes(4))
mat4_full = get_contingency_matrices(get_suit_sizes(4), reduce_perms=False)
mat4_full_full = get_contingency_matrices(get_suit_sizes(4, reduce=False), reduce_perms=False)
print(mat4.shape, mat4_full.shape, mat4_full_full.shape)
print(check_matrix_array(mat4))
print(check_matrix_array(mat4_full))
print(check_matrix_array(mat4_full_full))

(550727, 4, 3) (843818, 4, 3) (3265636, 4, 3)


100%|████████████████████████████████| 550727/550727 [00:39<00:00, 14004.30it/s]


3907043278


100%|████████████████████████████████| 843818/843818 [01:00<00:00, 13871.87it/s]


3916595178


100%|██████████████████████████████| 3265636/3265636 [03:54<00:00, 13911.21it/s]

3916595178


По очевидным причинам разница есть только, когда масти 3 или 4. Наибольшую выгоду даёт сортировка размеров мастей. А кодировать можно матрицы с любыми режимам сокращений.

In [15]:
from collections import defaultdict

def matrix_stats(all_matrices):
    moves = defaultdict(list)
    deals = defaultdict(list)
    for matrices in all_matrices:
        for matrix in matrices:
            hand_size = np.sum(matrix) // 3
            pct = PrefContingencyTable(matrix, True)
            moves[hand_size].append(pct.count_moves())
            deals[hand_size].append(pct.count_deals())
    return moves, deals

In [36]:
deals, moves = matrix_stats([mat1, mat2, mat3, mat4])

In [50]:
hand_size = 6
np.average(np.vstack(deals[hand_size]), axis=0, weights=moves[hand_size])

array([6.        , 2.21688445, 2.21192057])

In [5]:
pct = PrefContingencyTable(mat4[np.random.randint(mat4.shape[0])], True)
pct.matrix

array([[0, 6, 2],
       [2, 2, 1],
       [6, 0, 0],
       [1, 1, 6]])

In [7]:
pct.count_deals(), pct.count_moves(reduce=False)

(47040, array([9.        , 4.55555556, 2.        ]))

In [20]:
pct.count_deals(), pct.count_moves(reduce=True, suit_moves=suit_moves)

(47040, array([3.        , 1.33333333, 1.25      ]))

In [8]:
pct.trump

True

In [23]:
PrefContingencyTable(mat1[1], False).count_moves()

array([2., 2., 2.])

In [16]:
PrefContingencyTable(mat4[-1121], False).count_deals()

5644800

In [24]:
np.argwhere(mat4[-1121, :, 0] > 0)

array([[0],
       [2],
       [3]])

In [27]:
mat4[-1121]

array([[1, 2, 3],
       [0, 6, 2],
       [2, 2, 4],
       [7, 0, 1]])

In [36]:
mat4[-1121][np.where(mat4[-1121, :, 0] > 0)].cumprod(axis=0)

array([[ 1,  2,  3],
       [ 2,  4, 12],
       [14,  0, 12]])

In [31]:
m = np.copy(mat4[-1121])
m

array([[1, 2, 3],
       [0, 6, 2],
       [2, 2, 4],
       [7, 0, 1]])

In [33]:
m[np.where(m[:, 0] > 0)]

array([[1, 2, 3],
       [2, 2, 4],
       [7, 0, 1]])

In [9]:
get_contingency_matrices(get_suit_sizes(4), reduce_perms=False).shape

(843818, 4, 3)

In [40]:
np.allclose(mat2.reshape((-1, 3)), np.vstack(mat2)) 

True

#### Далее идёт что-то о подсчёте распределения числа взяток между руками при фиксированных размерах мастей

In [13]:
def perm2suit(perm, n_hands=3):
    result = [[] for i in range(n_hands)]
    for i, hand_index in enumerate(perm):
        result[hand_index].append(i)
    return result

class TrickCounter:
    def __init__(self, n_hands=3, reduce=True):
        self.trick = []
        self.reduce = reduce
        self.result = np.zeros(n_hands, dtype=np.int32)
        
    def __call__(self, suit):
        for i, card in enumerate(suit[0]):
            if self.reduce and i + 1 < len(suit[0]) and card + 1 == suit[0][i + 1]:
                continue
            self.trick.append(card)
            if len(suit) > 1:
                self(suit[1:])
            else:
                self.result[np.argmax(self.trick)] += 1
            self.trick.pop()
            
    def count(self):
        return self.result

class SuitTricksCounter:
    def __init__(self, suit_sizes, n_hands):
        self.suit_sizes = suit_sizes
        self.cards = list(range(sum(suit_sizes)))
        self.cards_by_hand = []
        self.total = np.zeros(n_hands, dtype=np.int32)
        self.total_reduced = np.zeros(n_hands, dtype=np.int32)
        
    def __call__(self, hands_in_play: list):
        for hand_cards in combinations(self.cards, self.suit_sizes[hands_in_play[0]]):
            self.cards_by_hand.append(list(hand_cards))
            self.cards = sorted(list(set(self.cards) - set(hand_cards)))
            if len(hands_in_play) > 1:    
                self(hands_in_play[1:])
            else:
                # print("Hands in play:", hands_in_play)
                # print("Cards by hand:", self.cards_by_hand)
                # print("Total shape:", self.total.shape)
                trick_counter = TrickCounter(len(self.cards_by_hand), reduce=False)
                trick_counter(self.cards_by_hand)
                self.total += trick_counter.count()
                trick_counter = TrickCounter(len(self.cards_by_hand), reduce=True)
                trick_counter(self.cards_by_hand)
                self.total_reduced += trick_counter.count()
            self.cards_by_hand.pop()
            self.cards.extend(hand_cards)
            self.cards = sorted(self.cards)
    
    def count(self):
        return self.total, self.total_reduced
        
            
def count_takers(suit_sizes: np.array, hands_in_play: list, reduce=True):
    hand_list = []
    for i in range(len(suit_sizes)):
        hand_list.extend(suit_sizes[i] * [i,])
    total = np.zeros(len(hands_in_play), dtype=np.int32)
    for perm in multiset_permutations(hand_list):
        suit = perm2suit(perm, len(hands_in_play))
        trick_counter = TrickCounter(len(hands_in_play), reduce=reduce)
        trick_counter([suit[i] for i in hands_in_play])
        total += trick_counter.count()
    return total

In [14]:
def count_reduced_moves(suit_sizes):
    result = np.zeros(len(suit_sizes))
    hand_list = []
    for i in range(len(suit_sizes)):
        hand_list.extend(suit_sizes[i] * [i,])
    total_perms = 0
    for perm in multiset_permutations(hand_list):
        total_perms += 1
        card_indices_by_hand = perm2suit(perm, len(suit_sizes))
        for i, card_indices in enumerate(card_indices_by_hand):
            for j, card in enumerate(card_indices):
                if j + 1 == len(card_indices) or card + 1 != card_indices[j + 1]:
                    result[i] += 1
    result /= total_perms
    return result

In [15]:
count_reduced_moves([2, 2, 1])

array([1.6, 1.6, 1. ])

In [26]:
hand_list = []
suit_sizes = [1, 6, 0]
for i in range(len(suit_sizes)):
    hand_list.extend(suit_sizes[i] * [i,])
for perm in multiset_permutations(hand_list):
    print(perm, perm2suit(perm))

[0, 1, 1, 1, 1, 1, 1] [[0], [1, 2, 3, 4, 5, 6], []]
[1, 0, 1, 1, 1, 1, 1] [[1], [0, 2, 3, 4, 5, 6], []]
[1, 1, 0, 1, 1, 1, 1] [[2], [0, 1, 3, 4, 5, 6], []]
[1, 1, 1, 0, 1, 1, 1] [[3], [0, 1, 2, 4, 5, 6], []]
[1, 1, 1, 1, 0, 1, 1] [[4], [0, 1, 2, 3, 5, 6], []]
[1, 1, 1, 1, 1, 0, 1] [[5], [0, 1, 2, 3, 4, 6], []]
[1, 1, 1, 1, 1, 1, 0] [[6], [0, 1, 2, 3, 4, 5], []]


In [19]:
s = [[] for i in range(3)]
s[1].append(2)
s

[[], [2], []]

In [112]:
suit_sizes = np.array([2, 2, 2], dtype=np.int8)
stc = SuitTricksCounter(suit_sizes, 2)
hands_in_play = [1, 2]
stc(hands_in_play)
print(stc.count())

(array([180, 180], dtype=int32), array([126, 126], dtype=int32))


In [21]:
suit_tricks = {}
suit_tricks_reduced = {}
for n in range(1, 9):
    for s in bounded_compositions(n, 2, [8, 8]):
        if s.min() > 0:
            # print(s)
            stc_2 = SuitTricksCounter(s, 2)
            stc_2([0, 1])
            suit_counts_2 = stc_2.count()
            stc_1 = SuitTricksCounter(s, 1)
            stc_1([1,])
            suit_counts_1 = stc_1.count()
            suit_tricks[tuple(s)] = (suit_counts_2[0][0], suit_counts_1[0][0])
            suit_tricks_reduced[tuple(s)] = (suit_counts_2[1][0], suit_counts_1[1][0])

In [22]:
for n in range(1, 9):
    for s in bounded_compositions(n, 3, [8, 8, 8]):
        if s.min() > 0:
            # print(s)
            stc_3 = SuitTricksCounter(s, 3)
            stc_3([0, 1, 2])
            suit_counts_3 = stc_3.count()
            
            stc_1_2 = SuitTricksCounter(s, 2)
            stc_1_2([1, 2])
            suit_counts_1_2 = stc_1_2.count()
            
            stc_1 = SuitTricksCounter(s, 1)
            stc_1([1,])
            suit_counts_1 = stc_1.count()
            
            stc_2 = SuitTricksCounter(s, 1)
            stc_2([2,])
            suit_counts_2 = stc_2.count()
            suit_tricks[tuple(s)] = (suit_counts_3[0][0], suit_counts_1_2[0][0],
                                     suit_counts_1[0][0], suit_counts_2[0][0])
            suit_tricks_reduced[tuple(s)] = (suit_counts_3[1][0], suit_counts_1_2[1][0],
                                             suit_counts_1[1][0], suit_counts_2[1][0])

In [24]:
for k, v in suit_tricks.items():
    print(k , v)

(1, 1) (1, 2)
(2, 1) (3, 3)
(1, 2) (3, 6)
(3, 1) (6, 4)
(2, 2) (12, 12)
(1, 3) (6, 12)
(4, 1) (10, 5)
(3, 2) (30, 20)
(2, 3) (30, 30)
(1, 4) (10, 20)
(5, 1) (15, 6)
(4, 2) (60, 30)
(3, 3) (90, 60)
(2, 4) (60, 60)
(1, 5) (15, 30)
(6, 1) (21, 7)
(5, 2) (105, 42)
(4, 3) (210, 105)
(3, 4) (210, 140)
(2, 5) (105, 105)
(1, 6) (21, 42)
(7, 1) (28, 8)
(6, 2) (168, 56)
(5, 3) (420, 168)
(4, 4) (560, 280)
(3, 5) (420, 280)
(2, 6) (168, 168)
(1, 7) (28, 56)
(1, 1, 1) (2, 3, 3, 3)
(2, 1, 1) (8, 6, 4, 4)
(1, 2, 1) (8, 12, 12, 4)
(1, 1, 2) (8, 12, 4, 12)
(3, 1, 1) (20, 10, 5, 5)
(2, 2, 1) (40, 30, 20, 5)
(1, 3, 1) (20, 30, 30, 5)
(2, 1, 2) (40, 30, 5, 20)
(1, 2, 2) (40, 60, 20, 20)
(1, 1, 3) (20, 30, 5, 30)
(4, 1, 1) (40, 15, 6, 6)
(3, 2, 1) (120, 60, 30, 6)
(2, 3, 1) (120, 90, 60, 6)
(1, 4, 1) (40, 60, 60, 6)
(3, 1, 2) (120, 60, 6, 30)
(2, 2, 2) (240, 180, 30, 30)
(1, 3, 2) (120, 180, 60, 30)
(2, 1, 3) (120, 90, 6, 60)
(1, 2, 3) (120, 180, 30, 60)
(1, 1, 4) (40, 60, 6, 60)
(5, 1, 1) (70, 21, 7, 7)


In [25]:
for k, v in suit_tricks_reduced.items():
    print(k , v)

(1, 1) (1, 2)
(2, 1) (2, 3)
(1, 2) (2, 4)
(3, 1) (3, 4)
(2, 2) (7, 9)
(1, 3) (3, 6)
(4, 1) (4, 5)
(3, 2) (15, 16)
(2, 3) (15, 18)
(1, 4) (4, 8)
(5, 1) (5, 6)
(4, 2) (26, 25)
(3, 3) (42, 40)
(2, 4) (26, 30)
(1, 5) (5, 10)
(6, 1) (6, 7)
(5, 2) (40, 36)
(4, 3) (90, 75)
(3, 4) (90, 80)
(2, 5) (40, 45)
(1, 6) (6, 12)
(7, 1) (7, 8)
(6, 2) (57, 49)
(5, 3) (165, 126)
(4, 4) (230, 175)
(3, 5) (165, 140)
(2, 6) (57, 63)
(1, 7) (7, 14)
(1, 1, 1) (2, 3, 3, 3)
(2, 1, 1) (6, 6, 4, 4)
(1, 2, 1) (6, 9, 9, 4)
(1, 1, 2) (6, 9, 4, 9)
(3, 1, 1) (12, 10, 5, 5)
(2, 2, 1) (26, 24, 16, 5)
(1, 3, 1) (12, 18, 18, 5)
(2, 1, 2) (26, 24, 5, 16)
(1, 2, 2) (26, 39, 16, 16)
(1, 1, 3) (12, 18, 5, 18)
(4, 1, 1) (20, 15, 6, 6)
(3, 2, 1) (68, 50, 25, 6)
(2, 3, 1) (68, 60, 40, 6)
(1, 4, 1) (20, 30, 30, 6)
(3, 1, 2) (68, 50, 6, 25)
(2, 2, 2) (142, 126, 25, 25)
(1, 3, 2) (68, 102, 40, 25)
(2, 1, 3) (68, 60, 6, 40)
(1, 2, 3) (68, 102, 25, 40)
(1, 1, 4) (20, 30, 6, 30)
(5, 1, 1) (30, 21, 7, 7)
(4, 2, 1) (140, 90, 36, 7)
(3, 3

In [18]:
suit_moves = {}
for n in range(1, 9):
    for s in bounded_compositions(n, 3, [8, 8, 8]):
        suit_moves[tuple(s)] = count_reduced_moves(s)

In [19]:
for k, v in suit_moves.items():
    print(k, v)

(1, 0, 0) [1. 0. 0.]
(0, 1, 0) [0. 1. 0.]
(0, 0, 1) [0. 0. 1.]
(2, 0, 0) [1. 0. 0.]
(1, 1, 0) [1. 1. 0.]
(0, 2, 0) [0. 1. 0.]
(1, 0, 1) [1. 0. 1.]
(0, 1, 1) [0. 1. 1.]
(0, 0, 2) [0. 0. 1.]
(3, 0, 0) [1. 0. 0.]
(2, 1, 0) [1.33333333 1.         0.        ]
(1, 2, 0) [1.         1.33333333 0.        ]
(0, 3, 0) [0. 1. 0.]
(2, 0, 1) [1.33333333 0.         1.        ]
(1, 1, 1) [1. 1. 1.]
(0, 2, 1) [0.         1.33333333 1.        ]
(1, 0, 2) [1.         0.         1.33333333]
(0, 1, 2) [0.         1.         1.33333333]
(0, 0, 3) [0. 0. 1.]
(4, 0, 0) [1. 0. 0.]
(3, 1, 0) [1.5 1.  0. ]
(2, 2, 0) [1.5 1.5 0. ]
(1, 3, 0) [1.  1.5 0. ]
(0, 4, 0) [0. 1. 0.]
(3, 0, 1) [1.5 0.  1. ]
(2, 1, 1) [1.5 1.  1. ]
(1, 2, 1) [1.  1.5 1. ]
(0, 3, 1) [0.  1.5 1. ]
(2, 0, 2) [1.5 0.  1.5]
(1, 1, 2) [1.  1.  1.5]
(0, 2, 2) [0.  1.5 1.5]
(1, 0, 3) [1.  0.  1.5]
(0, 1, 3) [0.  1.  1.5]
(0, 0, 4) [0. 0. 1.]
(5, 0, 0) [1. 0. 0.]
(4, 1, 0) [1.6 1.  0. ]
(3, 2, 0) [1.8 1.6 0. ]
(2, 3, 0) [1.6 1.8 0. ]
(1, 4, 0) [1.

In [ ]:
suit_pct.count_deals(), pct.count_moves(reduce=False)

In [25]:
suit_moves[(2, 2, 4)]

array([1.75, 1.75, 2.5 ])